# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [2]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(' '))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [3]:
import bz2
import csv
import pandas as pd

with bz2.open('ner-train.tsv.bz2', 'rt',encoding="utf8") as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open('ner-dev.tsv.bz2', 'rt',encoding="utf8") as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [4]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [5]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    P = len(set.intersection(gold,pred))/len(pred)
    
    r = len(set.intersection(gold,pred))/len(gold)
    
    F1 = 2*P*r/(P+r)
    
    print("Precision = ",round(P,2),"Recall = ",round(r,2),"F1 = ",round(F1,2))
    
    pass

To test your code, you can run the following cell:

In [6]:
evaluation_report(set(range(3)), set(range(5)))

Precision =  0.6 Recall =  1.0 F1 =  0.75


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [7]:
df_dev.head()

,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,2,3,Leicestershire_County_Cricket_Club
1,0946-001,LONDON 1996-08-30,0,1,London
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indies_cricket_team
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire_County_Cricket_Club


In [8]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    for row in df.itertuples():
        yield (row[1],row[3],row[4])

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [9]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [10]:
def pred(df):
    label = set()
    for i in range(len(df)):
        doc = nlp(df['sentence'][i])
        ents = ((df['sentence_id'][i], e.start, e.end) for e in doc.ents)
        label=label.union(ents)
    return label

labels = {((df_dev['sentence_id'][i], e.start, e.end) for e in nlp(df_dev['sentence'][i]).ents) 
         for i in range(len(df_dev))}

In [11]:
pred_label=pred(df_dev)

In [12]:
evaluation_report(spans_dev_gold,pred_label)

Precision =  0.54 Recall =  0.68 F1 =  0.6


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Here is a function that prints the false positives as well as the false negatives spans for a data frame, given a reference set of gold-standard spans and a candidate set of predicted spans.

In [13]:
from collections import defaultdict
import numpy as np

def error_report(df, spans_gold, spans_pred):
    FP = []
    FN = []
    false_pos = defaultdict(list)
    for s, b, e in spans_pred - spans_gold:
        false_pos[s].append((b, e))
    false_neg = defaultdict(list)
    for s, b, e in spans_gold - spans_pred:
        false_neg[s].append((b, e))
    for row in df.drop_duplicates('sentence_id').itertuples():
        if row.sentence_id in false_pos or row.sentence_id in false_neg:
            print('Sentence:', row.sentence)
            for b, e in false_pos[row.sentence_id]:
                print('  FP:', ' '.join(row.sentence.split()[b:e]))
                FP.append(row.sentence.split()[b:e])
            for b, e in false_neg[row.sentence_id]:
                print('  FN:', ' '.join(row.sentence.split()[b:e]))
                FN.append(row.sentence.split()[b:e])

    
    value,count = np.unique(FP, return_counts=True)
    
    top5 = ((value[count.argsort()[-10:]]),count[count.argsort()[-5:]])
    
    print('False positive report : ')
    
    print(top5)

Use this function to inspect and analyse the errors that the automated prediction makes. Can you see any patterns? Base your analysis on the first 500 rows of the training data. Summarize your observations in a short text.

In [14]:
df_train1=df_train[:500]

In [15]:
gold = gold_spans(df_train1)
gold = set(gold)
pred = pred(df_train1)

In [16]:
gold = set(gold_spans(df_train1))

In [17]:
error_report(df_train1,gold,pred)

Sentence: The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .
  FP: Thursday
  FP: The European Commission
  FN: European Commission
Sentence: Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .
  FP: Wednesday
  FP: the European Union 's
  FN: European Union
Sentence: He said further scientific study was required and if it was found that action was needed it should be taken by the European Union .
  FP: the European Union
  FN: European Union
Sentence: He said a proposal last month by EU Farm Commissioner Franz Fischler to ban sheep brains , spleens and spinal cords from the human and animal food chains was a highly specific and precautionary move to protect human health .
  FP: last month
  

In [18]:
label = set()
for i in range(len(df_train1)):
    doc = nlp(df_train1['sentence'][i])
    ents = ((e.text,e.label_) for e in doc.ents)
    label=label.union(ents)
    
for i in label:
    print(i)

('Shen Guofang', 'PERSON')
('Israeli-PLO', 'CARDINAL')
('KDP', 'ORG')
('labour ministry', 'ORG')
('EU', 'ORG')
('Syria', 'GPE')
('early next month', 'DATE')
('Russian', 'NORP')
('an estimated 800', 'CARDINAL')
('12 tonnes', 'QUANTITY')
('Hassan Hafidh', 'PERSON')
('Hizbollah', 'ORG')
('ATHENS', 'ORG')
('Israel Radio', 'ORG')
('Rabinovich', 'PERSON')
('the Russian Duma', 'ORG')
('10,925 pounds', 'QUANTITY')
('kingdom', 'GPE')
('Jordan', 'GPE')
('100', 'CARDINAL')
('GREEK SOCIALISTS', 'ORG')
('Chinese', 'NORP')
('BRUSSELS', 'GPE')
('this week', 'DATE')
('London Newsroom +44', 'ORG')
('a few days', 'DATE')
('24 hours', 'TIME')
('Athens Newsroom', 'ORG')
('Awdankiewicz', 'ORG')
('PUK', 'ORG')
('Lien Chan', 'PERSON')
('Monday', 'DATE')
('Iraqi Kurds', 'ORG')
('10', 'CARDINAL')
('Afghan UAE embassy', 'ORG')
('Arafat', 'PERSON')
('London', 'GPE')
('U.S.', 'GPE')
('Germany', 'GPE')
('UK', 'GPE')
('LONDON', 'GPE')
('Yasser Arafat', 'PERSON')
('Abu Dhabi', 'GPE')
('3,420', 'CARDINAL')
('1996-08-

Alteast one FPs is present in a sentence. Almost all the FPs. Most of these are days, time spans, random numbers denoting time or measurement also, these are psecific forms of the FNs.

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [19]:
def pred_better(df):
    label = set()
    for i in range(len(df)):
        doc = nlp(df['sentence'][i])
        ents = ((df['sentence_id'][i], e.start, e.end) 
                for e in doc.ents 
                if e.label_ !='DATE'
                if e.label_!='CARDINAL' 
                if e.label_!='TIME' 
                if e.label_!='MONEY'
               if e.label_!='ORDINAL'
               if e.label_!='PERCENT')
        label=label.union(ents)
    return label

In [20]:
pred_label_better = pred_better(df_dev)

In [21]:
for i in pred_label_better:
    print(i)

('1091-003', 42, 44)
('1022-003', 6, 7)
('0991-009', 1, 2)
('1156-009', 11, 12)
('0966-031', 4, 5)
('1008-010', 37, 38)
('1014-009', 13, 14)
('1016-003', 7, 10)
('1152-016', 8, 9)
('1030-004', 3, 4)
('1159-000', 7, 8)
('1094-030', 3, 4)
('1116-004', 8, 10)
('1161-002', 37, 38)
('0957-033', 12, 13)
('1096-013', 1, 2)
('1121-003', 22, 27)
('1099-003', 0, 1)
('1007-013', 34, 35)
('1115-009', 0, 1)
('1155-000', 8, 9)
('0948-033', 2, 5)
('0990-009', 11, 14)
('1094-011', 0, 1)
('1108-005', 0, 1)
('1125-008', 14, 15)
('1069-008', 0, 2)
('1058-004', 4, 5)
('1121-007', 32, 33)
('1099-007', 2, 4)
('1137-006', 0, 1)
('1099-018', 17, 19)
('0997-004', 14, 15)
('1042-007', 10, 11)
('0966-164', 10, 11)
('1156-008', 28, 29)
('0959-015', 0, 1)
('0974-007', 29, 31)
('1096-025', 12, 14)
('1056-034', 5, 7)
('0962-001', 2, 3)
('0966-034', 4, 5)
('0968-005', 40, 41)
('1019-003', 15, 16)
('1137-002', 26, 28)
('1005-015', 7, 9)
('1134-000', 2, 3)
('1106-001', 0, 1)
('1061-000', 0, 4)
('1072-019', 70, 74)
('09

In [22]:
evaluation_report(spans_dev_gold,pred_label_better)

Precision =  0.82 Recall =  0.68 F1 =  0.75


Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the *predicted* start and end positions for each token span, rather than the gold positions. As the `label` of each span, you can use the special value `--NME--`.

In [23]:
df = {'sentence_id': [pred_label[0] for pred_label in pred_label_better],
        'sentence': [" ".join(pd.unique(df_dev[df_dev['sentence_id'] == pred_label[0]]['sentence'])) 
                     for pred_label in pred_label_better ],
      'beg' : [pred_label[1] for pred_label in pred_label_better],
      'end' : [pred_label[2] for pred_label in pred_label_better],
      'label' : ['--NME--' for pred_label in pred_label_better ]
     }

In [24]:
df_dev_new= pd.DataFrame(df)

In [25]:
df_dev_new.head()

,sentence_id,sentence,beg,end,label
0,1091-003,"Women 's singles , third round 1 - Steffi Graf...",42,44,--NME--
1,1022-003,""" I think this morning 's Canadian numbers wer...",6,7,--NME--
2,0991-009,But Russia failed to win control over the whol...,1,2,--NME--
3,1156-009,Buyoya on Saturday dismissed its threat of an ...,11,12,--NME--
4,0966-031,2. Michael Johnson ( U.S. ) 20.02,4,5,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [26]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    for row in df.itertuples():
        yield (row[1],row[3],row[4],row[5])

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [27]:
wiki_tokens = ["_".join(row.sentence.split(" ")[row.beg:row.end]) for row in df_dev_new.itertuples()]

In [28]:
df_dev_new['label']= wiki_tokens

In [29]:
df_dev_new.head()

,sentence_id,sentence,beg,end,label
0,1091-003,"Women 's singles , third round 1 - Steffi Graf...",42,44,Petra_Langrova
1,1022-003,""" I think this morning 's Canadian numbers wer...",6,7,Canadian
2,0991-009,But Russia failed to win control over the whol...,1,2,Russia
3,1156-009,Buyoya on Saturday dismissed its threat of an ...,11,12,Burundi
4,0966-031,2. Michael Johnson ( U.S. ) 20.02,4,5,U.S.


In [30]:
gold = set(gold_mentions(df_dev))
pred_mentions = set(gold_mentions(df_dev_new))

In [31]:
evaluation_report(gold,pred_mentions)

Precision =  0.31 Recall =  0.26 F1 =  0.28


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on) as mentions of the entities (pages) that they link to. This allows us to harvest long lists of mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [32]:
with bz2.open('kb.tsv.bz2', 'rt',encoding="utf8") as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

In [33]:
df_kb.head()

,mention,entity,prob
0,000 Guineas,2000_Guineas_Stakes,1.0
1,10 00,United_States_dollar,1.0
2,126 million,United_States_dollar,1.0
3,13th dynasty,Middle_Kingdom_of_Egypt,1.0
4,14th Dalai Lama,14th_Dalai_Lama,1.0


To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [34]:
df_kb.loc[df_kb.mention == 'Sweden']

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing come first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [35]:
df_kb1 = df_kb.drop_duplicates(subset='mention', keep="first")
def entity_link(df):
    s_id = []
    s = []
    b = []
    e = []
    l = []
    for row in df_dev_new.itertuples():
        s_id.append(row.sentence_id)
        s1 = row.sentence.split(" ")
    #print(s1)
        s.append(row.sentence)
        b.append(row.beg)
        e.append(row.end)
    #print(s1[row.beg:row.end])
        if len(df_kb1.loc[df_kb1.mention == " ".join(s1[row.beg:row.end])]) != 0:
            temp = df_kb1.loc[df_kb1.mention == " ".join(s1[row.beg:row.end])]
            l.append(temp['entity'].tolist()[0])
        else:
            l.append('--NME--')
    df = pd.DataFrame(data = {"sentence_id" : s_id,
                             "sentence" : s,
                             "beg" : b,
                             "end" : e,
                             "label" : l})
    return(df)

In [36]:
df_dev_new2 = entity_link(df_dev_new)

In [37]:
df_dev_new2.head()

,sentence_id,sentence,beg,end,label
0,1091-003,"Women 's singles , third round 1 - Steffi Graf...",42,44,Petra_Langrová
1,1022-003,""" I think this morning 's Canadian numbers wer...",6,7,Canada
2,0991-009,But Russia failed to win control over the whol...,1,2,Russia
3,1156-009,Buyoya on Saturday dismissed its threat of an ...,11,12,Burundi
4,0966-031,2. Michael Johnson ( U.S. ) 20.02,4,5,--NME--


In [38]:
pred2 = set(gold_mentions(df_dev_new2))

In [39]:
evaluation_report(gold,pred2)

Precision =  0.63 Recall =  0.52 F1 =  0.57


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier that is trained to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [40]:
with bz2.open('contexts.tsv.bz2') as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [41]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in @ and the second in 1983
1,1970,FIFA_World_Cup,America 1975 and during the @ and 1978 World C...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the @
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the @ and ...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the @ The present capa...


Note that, in each context, the position of the mention is indicated by the `@` symbol.

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [42]:
df_contexts.context[df_contexts.mention == 'Lincoln']

41465    Nebraska Concealed Handgun Permit In @ municip...
41466    Lazlo restaurants are located in @ and Omaha C...
41467    California Washington Overland Park Kansas @ N...
41468    City Missouri Omaha Nebraska and @ Nebraska It...
41469    by Sandhills Publishing Company in @ Nebraska USA
                               ...                        
41609                                      @ Leyton Orient
41610                    English division three Swansea @ 
41611    league membership narrowly edging out @ on goa...
41612                                          @ Cambridge
41613                                                   @ 
Name: context, Length: 149, dtype: object

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [121]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [122]:
unique_mentions = df_contexts.drop_duplicates("mention")

In [123]:
classifier_dict = {}
for row in unique_mentions.itertuples(): 
    
    # Finding prior prob
    prior_prob = df_kb.loc[df_kb.mention == row.mention].sort_values("entity")["prob"].values
    
    # Making a pipeline 
    pipe =  Pipeline([
        ('vectorizer', CountVectorizer(lowercase=True)),
        ('classifier', MultinomialNB(class_prior=prior_prob))])
    
    # Fitting the pipeline
    pipe.fit(X = df_contexts.context[df_contexts.mention == row.mention],
              y = df_contexts.entity[df_contexts.mention == row.mention])
    
    classifier_dict[row.mention] = pipe

In [124]:
label = []

for row in df_dev_new.itertuples():
    
    #getiing the context
    context = ' '.join(row[2].split()[row[3]:row[4]])
     
    #Checking if the context is present in our classifier dict
    if context in classifier_dict:
        men = classifier_dict[context].predict([row[2]])[0]
        
    # Else taking the mention name from the mention data frame    
    elif context in df_kb["mention"].values:
        men = df_kb.loc[df_kb["mention"] == context, "entity"].values[0]
           
    else:
        men = "--NME--"
        
    label.append(men)

In [125]:
df_dev_new3 = df_dev_new.copy()
df_dev_new3['label']=label
df_dev_new3.head()

,sentence_id,sentence,beg,end,label
0,1091-003,"Women 's singles , third round 1 - Steffi Graf...",42,44,Petra_Langrová
1,1022-003,""" I think this morning 's Canadian numbers wer...",6,7,Canada
2,0991-009,But Russia failed to win control over the whol...,1,2,Russia
3,1156-009,Buyoya on Saturday dismissed its threat of an ...,11,12,Burundi
4,0966-031,2. Michael Johnson ( U.S. ) 20.02,4,5,--NME--


In [126]:
pred3 = set(gold_mentions(df_dev_new3))

In [127]:
evaluation_report(gold,pred3)

Precision =  0.64 Recall =  0.54 F1 =  0.58


We see an increase of precision, recall and F1 after using the context-sensitive disambiguation method.

You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1.

**This was the last lab in the Text Mining course. Congratulations!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>